<a href="https://colab.research.google.com/github/lauren-safwat/World-University-Rankings-Dashboard/blob/main/World_University_Rankings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install dash

In [2]:
!pip3 install jupyter_dash

In [ ]:
!pip3 install dash_bootstrap_components

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

import jupyter_dash
from dash import Dash, html, dcc
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc
from dash.exceptions import PreventUpdate
import plotly.express as px
import plotly.graph_objs as go
from datetime import date
from datetime import datetime

In [5]:
unis = pd.read_csv('/content/World_University_Rankings.csv')

In [7]:
app = jupyter_dash.JupyterDash(external_stylesheets=[dbc.themes.DARKLY])

In [8]:
app.layout = html.Div(children=[
    html.Div([
        html.H1('Ranking of Universities Around the World')
    ])
])

In [9]:
app.run_server()

Dash app running on:


<IPython.core.display.Javascript object>